In [19]:
import matplotlib.pyplot as plt
import pandas as pd
from glob import glob
from datetime import datetime
from IPython.display import display
%matplotlib inline 

In [2]:
arquivos = glob("C:/Users/ppgar/OneDrive/Documentos/Dados/Treino/*.csv")
treino = pd.DataFrame([])
for arquivo in arquivos:
    dados = pd.read_csv(arquivo, sep = ";")
    print(arquivo)
    treino = pd.concat([treino, dados])

C:/Users/ppgar/OneDrive/Documentos/Dados/Treino\GERACAO_USINA_2000.csv
C:/Users/ppgar/OneDrive/Documentos/Dados/Treino\GERACAO_USINA_2001.csv
C:/Users/ppgar/OneDrive/Documentos/Dados/Treino\GERACAO_USINA_2002.csv
C:/Users/ppgar/OneDrive/Documentos/Dados/Treino\GERACAO_USINA_2003.csv
C:/Users/ppgar/OneDrive/Documentos/Dados/Treino\GERACAO_USINA_2004.csv
C:/Users/ppgar/OneDrive/Documentos/Dados/Treino\GERACAO_USINA_2005.csv
C:/Users/ppgar/OneDrive/Documentos/Dados/Treino\GERACAO_USINA_2006.csv
C:/Users/ppgar/OneDrive/Documentos/Dados/Treino\GERACAO_USINA_2007.csv
C:/Users/ppgar/OneDrive/Documentos/Dados/Treino\GERACAO_USINA_2008.csv
C:/Users/ppgar/OneDrive/Documentos/Dados/Treino\GERACAO_USINA_2009.csv
C:/Users/ppgar/OneDrive/Documentos/Dados/Treino\GERACAO_USINA_2010.csv
C:/Users/ppgar/OneDrive/Documentos/Dados/Treino\GERACAO_USINA_2011.csv
C:/Users/ppgar/OneDrive/Documentos/Dados/Treino\GERACAO_USINA_2012.csv
C:/Users/ppgar/OneDrive/Documentos/Dados/Treino\GERACAO_USINA_2013.csv
C:/Use

In [3]:
treino['din_instante'] = pd.to_datetime(treino['din_instante'])
treino['data'] = treino['din_instante'].dt.strftime('%Y-%m')
#c

In [23]:
treino = treino.drop(['din_instante', 'nom_subsistema', 'nom_estado', 'ceg', 'id_ons', 'nom_tipousina'], axis = 1)

KeyError: "['din_instante', 'nom_subsistema', 'nom_estado', 'ceg', 'id_ons'] not found in axis"

In [16]:
agrupado = treino.groupby(['nom_tipocombustivel', 'data'])

In [22]:
display(agrupado.head())

,id_subsistema,id_estado,cod_modalidadeoperacao,nom_tipousina,nom_tipocombustivel,nom_usina,val_geracao,data
0,N,PA,TIPO I,HIDROELÉTRICA,Hidráulica,Tucuruí,2422.500,2000-01
1,NE,AL,TIPO I,HIDROELÉTRICA,Hidráulica,Xingó,1995.600,2000-01
2,NE,BA,Pequenas Usinas (Tipo III),HIDROELÉTRICA,Hidráulica,Pequenas Centrais Hidroelétricas da Chesf,13.000,2000-01
3,NE,BA,TIPO I,HIDROELÉTRICA,Hidráulica,Apolônio Sales,202.500,2000-01
4,NE,BA,TIPO I,HIDROELÉTRICA,Hidráulica,Paulo Afonso II,269.800,2000-01
...,...,...,...,...,...,...,...,...
3913677,SE,RJ,TIPO I,NUCLEAR,Nuclear,Angra 1,0.000,2018-12
3913678,SE,RJ,TIPO I,NUCLEAR,Nuclear,Angra 2,1367.760,2018-12
3914180,SE,RJ,TIPO I,NUCLEAR,Nuclear,Angra 1,0.000,2018-12
3914181,SE,RJ,TIPO I,NUCLEAR,Nuclear,Angra 2,1367.722,2018-12
